# 1. Import lab8.xlsx data

In [53]:
# To run in colab:
# %%capture
# !pip install bokeh
# !git clone https://github.com/sigord/data_visualisation/
# %cd /content/data_visualisation/LR8

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import numpy as np

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Select, HoverTool, Whisker
from bokeh.layouts import row, column
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap, factor_mark, linear_cmap, cumsum, jitter
from bokeh.palettes import Category20c

df=pd.read_excel('lab8.xlsx')

# 2. Prepare the data - create an average salary column between salary.from and salary.to. If only one marginal is available - take it as an estimate of the salary for this job.

In [55]:
df2 = df.copy()
df2['salary.average'] = df2.apply(lambda row: np.nan \
    if (np.isnan(row['salary.to']) and np.isnan(row['salary.from'])) \
        else \
            (row['salary.from'] \
        if np.isnan(row['salary.to']) \
            else (row['salary.to'] \
        if np.isnan(row['salary.from']) \
            else (row['salary.from'] + row['salary.to'])/2)), axis=1)

# 3. Prepare the data - select a new column from the job titles with the values June, midle, senor, use either Python built-in string functions or regular expressions. Consider both Russian and English spelling of these terms.

In [56]:
df2['level'] = df2.apply(lambda row: 'Junior' if re.search(r'(?i)junior|младший|джуниор', row['name']) \
        else ('Middle' if re.search(r'(?i)middle|средний|миддл', row['name']) \
        else ('Senior' if re.search(r'(?i)senior|старший|сеньор', row['name']) \
        else ('Team Lead' if re.search(r'(?i)team lead|тимлид', row['name']) \
        else ('Intern' if re.search(r'(?i)intern|стажер|стажёр|интерн', row['name']) \
        else ('Principal' if re.search(r'(?i)principal|принципал', row['name'])
        else 'Unknown'))))), axis=1)

# 4. Prepare data - leave only vacancies with average salary. Cleaned dataframe used in following tasks.

In [57]:
df2 = df2.dropna(subset=['salary.average'])

EUR_TO_RUB = 90
USD_TO_RUB = 80
df2['salary.average.rub'] = df2.apply(lambda row: row['salary.average'] if row['salary.currency'] == 'RUR' \
        else (row['salary.average']*USD_TO_RUB if row['salary.currency'] == 'USD' \
        else (row['salary.average']*EUR_TO_RUB if row['salary.currency'] == 'EUR' \
        else np.nan)), axis=1)
    

# 5. Create a bubble diagram in bokeh, where x-axis is town, y-axis is (June, Middle, senor), bubble size - number of vacancies, bubble colour - (the lighter the bubble, the higher the the higher the median salary, the darker the lower the median salary)

In [58]:
df5 = df2.groupby(['area.name', 'level']).size().reset_index(name='vacancies')

output_notebook()

df5['level'] = df5['level'].astype('category')
df5['area.name'] = df5['area.name'].astype('category')
color_mapper = CategoricalColorMapper(factors=df5['level'].cat.categories.tolist(), palette=Spectral6)
source = ColumnDataSource(df5)

p = figure(title="Vacancies by area and level", 
           x_range=df5['area.name'].cat.categories.tolist(), 
           y_range=df5['level'].cat.categories.tolist())
p.circle(x='area.name', 
         y='level', source=source, 
         size='vacancies', color={'field': 'level', 'transform': color_mapper}, 
         alpha=0.5)
p.add_tools(HoverTool(tooltips=[('Area', '@{area.name}'), ('Level', '@{level}'), ('Vacancies', '@vacancies')]))
p.title.text = 'Vacancies by area and level'
p.title.align = 'center'
layout = row(p)
show(layout)

Loading BokehJS ...

# 6. Create a horizontal bar graph with the town on the axis of ordinates and the number of vacancies on the axis of abscissa. xaxis - number of vacancies in the city.


In [59]:
df6 = df2.groupby(['area.name']).size().reset_index(name='vacancies')
df6['area.name'] = df6['area.name'].astype('category')
color_mapper = CategoricalColorMapper(factors=df6['area.name'].cat.categories.tolist(), palette=Spectral6)
source = ColumnDataSource(df6)
p = figure(title="Vacancies by area",
              y_range=df6['area.name'].cat.categories.tolist())
p.hbar(y='area.name',
       right='vacancies',
       source=source,
       height=0.5,
       color={'field': 'area.name', 'transform': color_mapper},
       alpha=0.5)
p.add_tools(HoverTool(tooltips=[('Area', '@{area.name}'), ('Vacancies', '@vacancies')]))
p.title.text = 'Vacancies by area'
p.title.align = 'center'
layout = row(p)
show(layout)

# 7. Create a pie chart - the number of vacancies with and without taxes (gross=True)


In [60]:
df7 = df2.groupby(['salary.gross']).size().reset_index(name='vacancies')

output_notebook()
df7['salary.gross'] = df7['salary.gross'].astype('bool')
df7['salary.gross'] = df7['salary.gross'].astype('category')
df7['salary.gross'] = df7['salary.gross'].cat.rename_categories(['Net', 'Gross'])
df7['angle'] = df7['vacancies']/df7['vacancies'].sum() * 2*np.pi

colors = ['#718dbf', '#e84d60']
df7['color'] = colors
source = ColumnDataSource(df7)
p = figure(title="Vacancies by gross",
           height=350,
           toolbar_location=None,
           x_range=(-0.5, 1.0))
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='salary.gross', source=source)
p.add_tools(HoverTool(tooltips=[('Type', '@{salary.gross}'), ('Vacancies', '@vacancies')]))
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
p.title.text = 'Vacancies by salary type'
p.title.align = 'center'
layout = row(p)
show(layout)

Loading BokehJS ...

# 8. Create a box chart - distribution of salary amounts by city.

In [92]:
df8 = df2[['area.name', 'salary.average.rub']]
df8.columns = ['area', 'salary']
df8['area'] = df8['area'].astype('category')

classes = df8['area'].cat.categories.tolist()

output_notebook()

g = df8.groupby('area')
q1 = g.quantile(q=0.25)
q2 = g.quantile(q=0.5)
q3 = g.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.salary > upper.loc[cat]['salary']) | (group.salary < lower.loc[cat]['salary'])]['salary']
out = g.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])
        
p = figure(tools="", background_fill_color="#efefef", x_range=classes, toolbar_location=None)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = g.quantile(q=0.00)
qmax = g.quantile(q=1.00)
upper.salary = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'salary']),upper.salary)]
lower.salary = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'salary']),lower.salary)]

# stems
p.segment(classes, upper.salary, classes, q3.salary, line_color="black")
p.segment(classes, lower.salary, classes, q1.salary, line_color="black")

# boxes
p.vbar(classes, 0.7, q2.salary, q3.salary, fill_color="#E08E79", line_color="black")
p.vbar(classes, 0.7, q1.salary, q2.salary, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(classes, lower.salary, 0.2, 0.01, line_color="black")
p.rect(classes, upper.salary, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"
p.title.text = 'Salary distribution by city'
p.title.align = 'center'
p.xaxis.axis_label = 'City'
p.yaxis.axis_label = 'Salary'
p.add_tools(HoverTool(tooltips=[('City', '@{area}'), ('Salary', '@{salary} rub')]))
layout = row(p)
show(layout)


/var/folders/k0/pk1b16mx61j0c8lv8m_l5gl40000gn/T/ipykernel_20766/882029753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['area'] = df8['area'].astype('category')


Loading BokehJS ...

# 9. Add chart titles for 5-8. Add a pop-up caption (city or availability of tax, or specialist qualifications) for each primitive in Charts 5-8.

In [ ]:
# Done

# 10. Draw a house in bokeh, with a triangular roof and a window, make the walls white, the roof in red. (done without sample data)

In [ ]:
output_notebook()
p = figure(width=400, height=400)
p.patch([1, 1, 3, 3], [1, 3, 3, 1], color='white')
p.patch([1, 2, 3], [3, 4, 3], color='red')
p.patch([1.5, 1.5, 2.5, 2.5], [1.5, 2.5, 2.5, 1.5], color='blue')
p.patch([1.97, 1.97, 2.03, 2.03], [1.5, 2.5, 2.5, 1.5], color='brown')
p.patch([1.5, 1.5, 2.5, 2.5], [1.97, 2.03, 2.03, 1.97], color='brown')
p.background_fill_color = "black"
show(p)

Loading BokehJS ...